# Web Scraping Highest-Grossing Films from Wikipedia
This notebook scrapes Wikipedia's **List of highest-grossing films**, extracting key details like title, release year, director(s), box office revenue, and country of origin.

### Supported Wikipedia Tables:
I target **four different tables** on the Wikipedia page, each with a unique structure:
1. **Highest-grossing films** (Default)
2. **Highest-grossing films adjusted for inflation**
3. **Highest-grossing films by year of release**
4. **Timeline of the highest-grossing film record**

By default, I parse only the first table, but the code can be configured to scrape all four tables.

In [1]:
# Imports
import asyncio
import re

import httpx
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [2]:
# Constants for parsing
WIKI_BASE_URL = 'https://en.wikipedia.org'
WIKI_TARGET_PAGE = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_films'
HEADERS = {'User-Agent': 'Mozilla/5.0'}

## Fetching the Wikipedia Page
I used `requests` (sync) and `httpx` (async) to fetch Wikipedia data.
- **`fetch_url(url)`** → Synchronous HTTP request.
- **`fetch_url_async(async_client, url)`** → Async request for parallel scraping.


In [3]:
def fetch_url(url):
    """Fetches a webpage synchronously using requests."""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()  # Handle HTTP errors (4xx/5xx)
        return response.text
    except requests.exceptions.HTTPError as e:
        print(f'Error fetching {url}: {e}')
        return None


async def fetch_url_async(async_client, url):
    """Fetches a webpage asynchronously using httpx or another async client."""
    try:
        response = await async_client.get(url)
        response.raise_for_status()  # Handle HTTP errors (4xx/5xx)
        return response.text
    except httpx.HTTPStatusError as e:
        print(f'Error fetching {url}: {e}')
        return None

## Extracting Film Links
The function `get_film_links(parse_all_links=False)` extracts links from the Wikipedia tables.
- Default: Parses only **Highest-grossing films**.
- If `parse_all_links=True`, it extracts from **all four tables**.


In [4]:
def get_film_links(parse_all_links: bool = False) -> set[str]:
    """
    Extracts Wikipedia film page links from the 'List of highest-grossing films' page.
    
    Args:
        parse_all_links (bool): 
            - False (default) → Parses only the first table (highest-grossing films).
            - True → Parses all four tables.
    
    Returns:
        set[str]: A set of unique Wikipedia URLs for films.
    """
    
    page_content = fetch_url(WIKI_TARGET_PAGE)
    
    if not page_content:
        return set()
    
    soup = BeautifulSoup(page_content, 'lxml')

    # Select tables based on the 'parse_all_links' flag
    if parse_all_links:
        tables = soup.select('table.wikitable.plainrowheaders')  # Get all tables
    else:
        tables = [soup.select_one('table.wikitable.plainrowheaders')]  # Get only the first table
    
    film_links = set()

    # Loop through each table and extract film links
    for i, table in enumerate(tables):
        for row in table.select('tbody tr'):

            # Handling different table structures:
            if i < 2:  # First two tables: Film links are in <th scope="row"><i><a href="...">
                th = row.find('th', scope='row')
                link = th.find('i').find('a') if th and th.find('i') else None
            else:  # Last two tables: Film links are in <td><i><a href="...">
                td = row.find('td')
                link = td.find('i').find('a') if td and td.find('i') else None

            if link and 'href' in link.attrs:
                film_links.add(urljoin(WIKI_BASE_URL, link['href']))
    
    return film_links

## Asynchronous Film Data Extraction
To speed up scraping, I fetch film pages asynchronously using `httpx.AsyncClient()`.

Each film page is parsed using `parse_film_page()`, which extracts relevant details from the `infobox`.

## Cleaning and Structuring Data
To ensure clean data storage, I perform several transformations:
- **Extract only the release year** from messy date formats.
- **Remove references (`[1]`, `[2]`)** from text fields.
- **Normalize country and director names** into a readable format.


In [5]:
async def parse_film_page(async_client, url):
    """
    Asynchronously fetches and parses a Wikipedia film page to extract key details.

    Args:
        async_client (httpx.AsyncClient): The async HTTP client for fetching the page.
        url (str): The Wikipedia URL of the film.

    Returns:
        dict or None: A dictionary containing film details (title, release year, director, box office, country, url)
                      or None if the page could not be parsed.
    """
    
    page_content = await fetch_url_async(async_client, url)
    
    if not page_content:
        return None

    # Locate the film's infobox (table containing key details)
    soup = BeautifulSoup(page_content, 'lxml')
    info_box = soup.select_one('table.infobox.vevent tbody')
    
    if not info_box:
        return None

    # Extracts text from an info box row given its label.
    def get_info(label):
        row = info_box.find('th', string=label)
        if row and row.find_next_sibling('td'):
            return row.find_next_sibling('td').get_text(', ', strip=True)
        return None

    # Extracts the four-digit release year from a date string.
    def extract_year(date_text):
        match = re.search(r'\b(\d{4})\b', date_text)
        return int(match.group(1)) if match else None
    
    # Cleans and formats a string containing multiple values (e.g., directors, countries).
    def clean_multivalued(value):
        value = re.sub(r'\[.*?\]', '', value)  # Removes reference numbers (e.g., [1], [2])
        value = re.sub(r'\(.*?\)', '', value)  # Removes information in parentheses
        
        # Splits values by commas and trims whitespace
        value_list = [val for val in value.split(', ') if val not in ('', ' ')]
        value = ', '.join(value_list).strip(', ')
        return value
    
    # Cleans and formats the box office revenue.
    def clean_box_office(value):
        value = re.sub(r'\[.*?\]', '', value)  # Removes reference numbers
        return value.replace('\xa0', ' ').replace(',', '').strip(', ')

    # Extract key details from the infobox
    title = info_box.find('tr').text
    release_year = get_info('Release date') or get_info("Release dates") or get_info('Released')
    director = get_info('Directed by')
    box_office = get_info('Box office').replace('US', '')
    country = get_info('Country') or get_info('Countries')

    # Store the extracted details in a dictionary
    film_data = {
        'title': title,
        'release_year': extract_year(release_year),
        'director': clean_multivalued(director) if director else None,
        'box_office': clean_box_office(box_office) if box_office else None,
        'country': clean_multivalued(country) if country else None,
        'url': url
    }

    # Return the film data only if a title was successfully extracted
    return film_data if film_data.get('title') else None

In [6]:
async def parse_film_data(film_links):
    """
    Asynchronously fetches and parses multiple film pages.
    
    Args:
        film_links (list[str]): A list of Wikipedia film URLs.
    
    Returns:
        list[dict]: A list of parsed film data dictionaries.
    """
    async with httpx.AsyncClient() as client:
        tasks = [parse_film_page(client, url) for url in film_links]
        results = await asyncio.gather(*tasks)
        films = [film for film in results if film]
    
    return films

In [7]:
# Get film urls by parsing target wiki page
links = list(get_film_links())

## Running Async Parsing in Jupyter
Since Jupyter runs an event loop, `asyncio.run()` **cannot be used directly**.

`await parse_film_data(links)` **works fine in IPython environments (Jupyter, Colab, Kaggle, DataSpell).**

If running in standard Jupyter, use:
```python
import nest_asyncio
nest_asyncio.apply()
```

In [8]:
# Parse all data from film urls
data = await parse_film_data(links)
data.sort(key=lambda film: film.get('box_office', 0), reverse=True)

## Saving Data to Database
The extracted data is stored in an SQLite database. I ensure:
- Unique constraints to prevent duplicate entries.
- Proper data types (`INT` for year, `TEXT` for other fields).


In [9]:
# Imports
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker, declarative_base

In [10]:
# Database file path (SQLite database)
DB_URL = 'sqlite:///grossing_films.db'

In [11]:
# Create a database engine
engine = create_engine(DB_URL)

# Base class for ORM models
Base = declarative_base()


class Film(Base):
    """
    SQLAlchemy ORM model representing a film record.
    
    Attributes:
        id (int): Primary key, auto-incremented.
        title (str): The film's title (required).
        release_year (int): Year of release.
        director (str): Name(s) of the director(s).
        box_office (str): Box office revenue.
        country (str): Country of origin.
        url (str): Wikipedia URL for reference.
    """
    __tablename__ = 'films'
    id = Column(Integer, primary_key=True)
    title = Column(String, nullable=False)
    release_year = Column(Integer)
    director = Column(String)
    box_office = Column(String)
    country = Column(String)
    url = Column(String)

# Create the database table (if it doesn't already exist)
Base.metadata.create_all(engine)

# Create a session for interacting with the database
Session = sessionmaker(bind=engine)
session = Session()

# Remove existing records to avoid duplicates before inserting new ones
session.query(Film).delete()

# Add all film records to the session and commit transaction
film_rows = [Film(**film) for film in data]
session.add_all(film_rows)
session.commit()

## Saving Data to JSON
The extracted data is exported to JSON file for further usage in frontend.

In [12]:
import json

# Save the scraped data as a JSON file
with open('grossing_films.json', 'w') as f:
    json.dump(data, f)